In [1]:
import torch
import torchvision
from torch import nn
from torchvision import transforms

print(f"torch {torch.__version__} | torchvision {torchvision.__version__}")

torch 2.0.1+cu118 | torchvision 0.15.2+cu118


In [2]:
# import model summary from torchinfo
try:
    from torchinfo import summary
except:
    print(f"[INFO] installing torchinfo...")
    !pip install torchinfo
    from torchinfo import summary

In [3]:
# import modules from sources
from sources import utils, datasetup, engine, models

2023-11-07 15:54:47.104179: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-07 15:54:47.148906: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-07 15:54:47.860466: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
# device agnostic
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [5]:
# downloading data (10 and 20% training data)
source_10 = "https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip"
source_20 = "https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi_20_percent.zip"

image_path_10 = datasetup.download_data(source=source_10, destination="image_10")
image_path_20 = datasetup.download_data(source=source_20, destination="image_20")
image_path_10, image_path_20

[INFO] creating data/image_10...
[INFO] downloading pizza_steak_sushi.zip to data...
[INFO] extracting pizza_steak_sushi.zip to data/image_10...
[INFO] removing pizza_steak_sushi.zip...
[INFO] creating data/image_20...
[INFO] downloading pizza_steak_sushi_20_percent.zip to data...
[INFO] extracting pizza_steak_sushi_20_percent.zip to data/image_20...
[INFO] removing pizza_steak_sushi_20_percent.zip...


(PosixPath('data/image_10'), PosixPath('data/image_20'))

In [6]:
# setup train and test directories
train_dir_10 = image_path_10 / "train"
train_dir_20 = image_path_20 / "train"

# use the same test dir for both datasets
test_dir = image_path_10 / "test"

train_dir_10, train_dir_20, test_dir

(PosixPath('data/image_10/train'),
 PosixPath('data/image_20/train'),
 PosixPath('data/image_10/test'))

In [7]:
# training grid
effnet_version_list = [0,1,2]
train_dir_list = [train_dir_10, train_dir_20]
epochs_list = [5, 10]

In [8]:

# create training kernel
def training_kernel(effnet_version: str,
                   train_dir: str,
                   epochs:int,
                   batch_size: int,
                   lr: float,
                   device: torch.device,
                   model_dir: str):


    grid_name_1 = f"effnet{effnet_version}"
    grid_name_2 = str(train_dir).replace("/","_")
    grid_name_3 = f"epochs_{epochs}"

    # create model and model transforms
    model, model_transforms = models.create_effnet(effnet_version=effnet_version, INFO=False)

    # create dataloaders
    train_dataloader, test_dataloader, class_names = datasetup.create_dataloaders(
        train_dir = train_dir,
        test_dir = test_dir,
        train_transforms=model_transforms,
        test_transforms=model_transforms,
        batch_size=batch_size,
        INFO=False
    )
    # create loss function and optimizer
    loss_fn = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(params=model.parameters(), lr=lr)

    # grid list
    grid_name_list = [model.name, grid_name_2, grid_name_3]
    
    # perform training grid
    results = engine.train_tsb_writer(
        model = model,
        train_dataloader = train_dataloader,
        test_dataloader = test_dataloader,
        loss_fn = loss_fn,
        optimizer = optimizer,
        epochs=epochs,
        device=device,
        writer=utils.create_writer(grid_names=grid_name_list)
    )

    # save model
    model_path = utils.save_model(
        model=model,
        model_dir=model_dir,
        model_name_grid=grid_name_list
    )



In [9]:
BATCH_SIZE = 32

# perform training grid
experiment_num = 0

for effnet_version in effnet_version_list:
    for train_dir in train_dir_list:
        for epochs in epochs_list:
            
            experiment_num += 1
            print("\n")
            print(f"experiment number: {experiment_num} {'***'}")
            
            training_kernel(
                effnet_version=effnet_version,
                train_dir=train_dir,
                epochs=epochs,
                batch_size=BATCH_SIZE,
                lr=0.001,
                device=device,
                model_dir="saved_models/"
            )
            



experiment number: 1 ***
[INFO] creating writer log at: writerLog/EfficientNet_B0/data_image_10_train/epochs_5


  0%|          | 0/5 [00:00<?, ?it/s]

epoch: 0 | train_loss: 1.1175 | train_acc: 0.3359 | test_loss: 0.8715 | test_acc: 0.7538
epoch: 1 | train_loss: 0.9008 | train_acc: 0.6250 | test_loss: 0.7384 | test_acc: 0.8958
epoch: 2 | train_loss: 0.8019 | train_acc: 0.7070 | test_loss: 0.6693 | test_acc: 0.8864
epoch: 3 | train_loss: 0.6724 | train_acc: 0.9102 | test_loss: 0.6634 | test_acc: 0.8665
epoch: 4 | train_loss: 0.6572 | train_acc: 0.8125 | test_loss: 0.6225 | test_acc: 0.8561


experiment number: 2 ***
[INFO] creating writer log at: writerLog/EfficientNet_B0/data_image_10_train/epochs_10


  0%|          | 0/10 [00:00<?, ?it/s]

epoch: 0 | train_loss: 1.0668 | train_acc: 0.3477 | test_loss: 0.8216 | test_acc: 0.8343
epoch: 1 | train_loss: 0.8736 | train_acc: 0.8047 | test_loss: 0.7240 | test_acc: 0.8049
epoch: 2 | train_loss: 0.7911 | train_acc: 0.7695 | test_loss: 0.6675 | test_acc: 0.8665
epoch: 3 | train_loss: 0.7322 | train_acc: 0.7578 | test_loss: 0.6536 | test_acc: 0.8248
epoch: 4 | train_loss: 0.5912 | train_acc: 0.8867 | test_loss: 0.5522 | test_acc: 0.8655
epoch: 5 | train_loss: 0.6206 | train_acc: 0.7422 | test_loss: 0.5126 | test_acc: 0.8551
epoch: 6 | train_loss: 0.5227 | train_acc: 0.9180 | test_loss: 0.5097 | test_acc: 0.8352
epoch: 7 | train_loss: 0.5559 | train_acc: 0.8320 | test_loss: 0.5729 | test_acc: 0.7841
epoch: 8 | train_loss: 0.4763 | train_acc: 0.9375 | test_loss: 0.5032 | test_acc: 0.8248
epoch: 9 | train_loss: 0.4698 | train_acc: 0.8359 | test_loss: 0.4844 | test_acc: 0.8248


experiment number: 3 ***
[INFO] creating writer log at: writerLog/EfficientNet_B0/data_image_20_train/epochs

...

epoch: 0 | train_loss: 1.1275 | train_acc: 0.3555 | test_loss: 0.9757 | test_acc: 0.6013
epoch: 1 | train_loss: 0.9085 | train_acc: 0.6562 | test_loss: 0.8783 | test_acc: 0.7235
epoch: 2 | train_loss: 0.8135 | train_acc: 0.7305 | test_loss: 0.7599 | test_acc: 0.8561
epoch: 3 | train_loss: 0.6724 | train_acc: 0.8945 | test_loss: 0.6337 | test_acc: 0.9176
epoch: 4 | train_loss: 0.7091 | train_acc: 0.6758 | test_loss: 0.5693 | test_acc: 0.9176
epoch: 5 | train_loss: 0.6064 | train_acc: 0.7734 | test_loss: 0.6179 | test_acc: 0.8873
epoch: 6 | train_loss: 0.5713 | train_acc: 0.8086 | test_loss: 0.5792 | test_acc: 0.9384
epoch: 7 | train_loss: 0.4932 | train_acc: 0.9492 | test_loss: 0.5385 | test_acc: 0.8873
epoch: 8 | train_loss: 0.5598 | train_acc: 0.8125 | test_loss: 0.5462 | test_acc: 0.8873
epoch: 9 | train_loss: 0.4599 | train_acc: 0.9336 | test_loss: 0.5216 | test_acc: 0.8977


experiment number: 11 ***
[INFO] creating writer log at: writerLog/EfficientNet_B2/data_image_20_train/epoch

  0%|          | 0/5 [00:00<?, ?it/s]

epoch: 0 | train_loss: 0.9889 | train_acc: 0.5583 | test_loss: 0.7666 | test_acc: 0.9167
epoch: 1 | train_loss: 0.7056 | train_acc: 0.8083 | test_loss: 0.5883 | test_acc: 0.9375
epoch: 2 | train_loss: 0.6016 | train_acc: 0.8604 | test_loss: 0.5479 | test_acc: 0.8873
epoch: 3 | train_loss: 0.5450 | train_acc: 0.8167 | test_loss: 0.4385 | test_acc: 0.9479
epoch: 4 | train_loss: 0.4199 | train_acc: 0.9104 | test_loss: 0.4738 | test_acc: 0.8977


experiment number: 12 ***
[INFO] creating writer log at: writerLog/EfficientNet_B2/data_image_20_train/epochs_10


  0%|          | 0/10 [00:00<?, ?it/s]

epoch: 0 | train_loss: 0.9587 | train_acc: 0.6000 | test_loss: 0.7711 | test_acc: 0.7746
epoch: 1 | train_loss: 0.7341 | train_acc: 0.7917 | test_loss: 0.6101 | test_acc: 0.8873
epoch: 2 | train_loss: 0.5439 | train_acc: 0.8833 | test_loss: 0.5438 | test_acc: 0.9081
epoch: 3 | train_loss: 0.4763 | train_acc: 0.9021 | test_loss: 0.4726 | test_acc: 0.9081
epoch: 4 | train_loss: 0.4174 | train_acc: 0.9083 | test_loss: 0.4330 | test_acc: 0.8873
epoch: 5 | train_loss: 0.3709 | train_acc: 0.9104 | test_loss: 0.4002 | test_acc: 0.8977
epoch: 6 | train_loss: 0.3363 | train_acc: 0.9250 | test_loss: 0.3860 | test_acc: 0.8873
epoch: 7 | train_loss: 0.3498 | train_acc: 0.9021 | test_loss: 0.3507 | test_acc: 0.8873
epoch: 8 | train_loss: 0.2742 | train_acc: 0.9333 | test_loss: 0.3183 | test_acc: 0.9280
epoch: 9 | train_loss: 0.2729 | train_acc: 0.9375 | test_loss: 0.3474 | test_acc: 0.8977


In [10]:
# %load_ext tensorboard
# %tensorboard --logdir writerLog/ --port 0